In [ ]:
import sys
sys.path.insert(0,'../libs')
import fourier as foo
import ecualizacion as equ
from matplotlib import pyplot as plt
import numpy as np
from scipy import fftpack
from scipy import misc
# import seaborn as sns
# sns.despine()
from numpy import linalg
from scipy.signal import convolve2d
import random

In [ ]:
def convolve(I, K):
    return convolve2d(I,K,mode="same")

In [ ]:
def show_img(IMG, fig_size=(18,10)):
    plt.figure(figsize=fig_size); plt.axis('off')
    plt.imshow(IMG ,cmap='gray')
    plt.show()
    
def show_img_s(IMG, titles=None, fig_size = (18,10)):
    img_qty = len(IMG)
    plt.figure(figsize=fig_size); 
    for i in range(img_qty):
        plt.subplot(1,img_qty, i+1)
        if titles != None:
            plt.title(titles[i])
        plt.imshow(IMG[i] ,cmap='gray'); plt.axis('off')
    plt.tight_layout()
    plt.show()

## Obtención kernel pasa alto por inversa

**freq_filter** toma como parámetros la imágen a filtrar, y una función de decisión. La misma, en pos de la *distancia* de una cierta amplitud para una frecuencia f al centro del espacio, decide si será removida o no.

In [ ]:
def freq_filter(img, CUTOFF_FUN):
    filtered_img = np.copy(img)
    m = len(img); n = len(img[0])
    distance = lambda i,j: np.sqrt(abs(i-m/2)**2 + abs(j-n/2)**2)
    for i in range(m):
        for j in range(n):
            if not CUTOFF_FUN(distance(i,j)):
                filtered_img[i][j] = 0
    return filtered_img

In [ ]:
img = misc.imread('lena.png')

**Nota:** Si en algun momento se reodernana los cuadrantes de la imagen en el dominio de las frecuencias, antes de ser convertido por la IFFT, los mismos deben tener el mismo orden; ya sea el original, o con el el (0,0) de las frecuencias centrado en la imagen. En el *code-block* de abajo se puede ver ejemplos de ciertos filtros. En cada uno, **l** y **h** hacen referencia a los valores de corte inferior y superior.

In [ ]:
BAND_PASS_FILTER = lambda l,h: lambda d: (d <= l) or (d> h)
LOW_PASS_FILTER = lambda l: lambda d: (d <= l)
HIGH_PASS_FILTER = lambda h: lambda d: (d > h)

In [ ]:
img_F = fftpack.fft2(img)
# Aca ya reordeno los valores en complejo de toda la imagen (parte real e imaginaria),
# de forma de operar directamente con ellos, sin preocuparme.
img_F = foo.fix_norm_plot_regions(img_F)
img_ABS = np.abs(img_F)
# funcion de corte
CUTOFF_FUNC = HIGH_PASS_FILTER(50)
# aplico el filtro a la transformada
img_ABS = freq_filter(img_ABS, CUTOFF_FUNC)
# agregego los módulos filtrados con sus correspondientes ángulos
img_MOD = foo.assemble_complex(img_ABS, \
                               np.angle(img_F))
# anti transformo
img_deMOD = fftpack.ifft2(img_MOD)
show_img_s([img, foo.log_transform(img_ABS), np.abs(img_deMOD)],\
          ["original", "transformada filtrada", "antritransformada filtrada"])

La idea consiste en la siguiente conversión:

<p style="text-align: center;">

$\mathcal{F}(img)  \mathcal{F}(kernel) = \mathcal{F}(img_{filtered})$

$\mathcal{F}(img)^{-1}  \mathcal{F}(img)  \mathcal{F}(kernel) = \mathcal{F}(img)^{-1}  \mathcal{F}(img_{filtered})$

$\mathcal{F}(kernel) = \mathcal{F}(img)^{-1}  \mathcal{F}(img_{filtered})$

$\mathcal{F}(kernel) \xrightarrow[]{IDFT} kernel$

</p>
Pero el problema surge que para capturar una parte representativa del kernel, debo agrander su dimensión. Y esto solo sucede al realizar **lowPassFiltering**; en el caso contrario, no se logra el efecto deseado al aplicar el filtro resultante por convolución.

In [ ]:
# en esta operación realizo
"""
inversa(TRANSF(imagen original)) . TRANSF(imagen filtrada)
reordeno las regiones
antiTRANSFORMO
tomo muestra del kernel
"""
KERN_F = foo.fix_norm_plot_regions(np.dot(linalg.inv(img_F),img_MOD))
KERN = np.abs(fftpack.ifft2(KERN_F))
# NOTA: La antitransformada tiene el histrograma con un log-transform para hacer más visible los valores bajos
show_img_s([foo.log_transform(np.abs(KERN_F)), foo.log_transform(KERN)],\
           ["transformada del kernel obtenido", "antritransformada del kernel"])

In [ ]:
kernel_size_s = [10,20,30,50,100]
KERN_s = [KERN[0:i,0:i] for i in kernel_size_s]
KERN_s_labels = ["kernel size = " + str(i) for i in kernel_size_s]
CONV_s_labels = ["lena conv con k_size = " + str(i) for i in kernel_size_s]

show_img_s([foo.log_transform(k) for k in KERN_s], KERN_s_labels)
show_img_s([convolve(img, k) for k in KERN_s], CONV_s_labels)

Pruba de realizar el producto $\mathcal{F}(img)  \mathcal{F}(kernel)$ para ver si da como resultado $\mathcal{F}(img_{filtered})$ y $img_{filtered}$.

In [ ]:
filtered_img = np.dot(img_F, foo.fix_norm_plot_regions(KERN_F))
# print(filtered_img.shape)
show_img_s([foo.log_transform(np.abs(filtered_img)),\
            (np.abs(fftpack.ifft2(filtered_img)))],\
          ["Transformada del producto", "antitransformada del producto"])